?  =  variable name, possibly with indexing/slicing  
??? = function / class name  
?????? = complex expression containing variables, functions etc  
???B??? = complex expression that must contain "B"  

# Using behavior to predict neural activity

These groups of neurons that are not stimulus-selective, what are they doing? Maybe they are related to the behavior of the mouse?

If that's the case we need some low-dimensional representation of the behavior - we can't predict using 1 million pixels of video data - that's an underconstrained problem.

# 1. Load in data + behavior

In [ ]:
# this cell imports all the functions and data we need
import os # os stands for "operating system" and includes read/write routines etc. 
import numpy as np # by far the most used library for everyday computation
from scipy import io # this is for importing Matlab data files
from scipy import stats # here we import a whole sub-library of stats functions
from scipy.ndimage import gaussian_filter # here we import a single function
from sklearn.decomposition import PCA # check out all the other dimensionality reduction methods in "decomposition"
from matplotlib import pyplot as plt # all of our plotting is done with plt
from scipy.stats import zscore
%matplotlib inline 

# folder where data is 
root = '/home/neuraldata/data/meso/'
mouse_name = 'TX39'

spks = np.load(os.path.join(root, mouse_name, 'suite2p/combined/spks.npy'))
stat = np.load(os.path.join(root, mouse_name, 'suite2p/combined/stat.npy'), allow_pickle=True)
print('total neurons %d'%len(stat))

S = zscore(spks,axis=1)
del spks # delete spks to clear memory

Load in behavioral data

In [ ]:
# BEHAVIORAL DATA
proc   = np.load(os.path.join(root, mouse_name, 'cam1_TX39_2019_05_31_1_proc_resampled.npy'), allow_pickle=True)
proc   = proc.item() # non-array variables need to be accessed this way

In [ ]:
# what are the keys in proc? 
proc.keys()

In [ ]:
# show the first spatial motion SVD (and second, etc)
plt.imshow(proc['motMask'][?], cmap='bwr', vmin=-.02, vmax=.02)

In [ ]:
motSVD = proc['motSVD']
parea  = proc['pupil']['area'][0].copy()

# plot the first 1000 samples of the motion SVD
plt.subplot(2,1,1)
plt.plot(???)

# same for pupil area
plt.subplot(2,1,2)
plt.plot(???)
plt.show()

# Bin neural and behavioral data + compute neural PCA

We will bin the neural activity to do the PCA and the behavior (1 second bins)

In [ ]:
# bin data
tbin = int(3)
NT = motSVD.shape[1]
nt = int(np.floor(NT/tbin))
motSVD -= motSVD.mean(axis=1)[:,np.newaxis]
parea  -= np.nanmean(parea)
parea[np.isnan(parea)] = 0

beh = np.reshape(motSVD[:,:nt*tbin], (motSVD.shape[0], nt, tbin)).mean(axis=-1)
pup = np.reshape(parea[:nt*tbin], (nt, tbin)).mean(axis=-1)

# using this format bin S
# Sbin = ?
Sbin = np.reshape(S[:,:nt*tbin], (S.shape[0], nt, tbin)).mean(axis=-1)

Take principal components of neural data (as previously)

In [ ]:
from sklearn.decomposition import PCA

# use PCA to get the top 256 neural components. The components must be spatial (by neurons)
model_pca = ???256??? # 
U = model_pca.components_ # time components

# check the shape
print(U.shape)

# check that the Us are normalized to unit norm
plt.plot(np.sum(U**2, axis=?))

In [ ]:
V = U???Sbin # now project the binned data onto the spatial components. Figure out which transpose works! 

# Prediction of neural PCs

Now let's predict these neural PC's using our behavioral measures. But FIRST we need to split the data into train and test segments.

In [ ]:
# split into train-test
# * use interleaved segments *
nlen   = 500 
itrain = np.remainder(np.arange(nt), nlen) < ? # make 70% of the trials be training trials
itest  = ~itrain # what is not train is test

plt.plot(itrain)
plt.show()

Can you determine how I did the split above? Why might I have split into segments rather than randomly interleaving time-points?

# Prediction with behavioral variables

Use linear regression to perform the prediction, predict $Y$ using $X$:

$$ A = (X_\text{train}X_\text{train}^\top)^{-1} (X_\text{train} Y_\text{train}^\top)$$

$X$ is behavioral components by time, $Y$ is neural components by time. If you want to L2 regularize the linear regression:

$$ A = (X_\text{train}X_\text{train}^\top + \lambda I)^{-1} (X_\text{train} Y_\text{train}^\top)$$

Then the prediction on time points is:

$$ \hat Y_\text{test} = A^\top X_\text{test} $$

In [ ]:
#### PREDICT USING PUPIL WITH LINEAR REGRESSION

X = pup[itrain]
COV = ?????? # determine the covariance of the one-dimensional input X
print(COV)

A = 1/COV * (pup[?] @ V[:,?]) # multiply training timepoints of pupil and neural PCs

In [ ]:
# what is the explained variance on test timepoints?
Xtest  = pup[itest]
Vpredp = A[:,np.newaxis] * Xtest[np.newaxis,:] # predict on test points

resid  = ? - ? # what are the unexplained residuals of the prediction? 

varexp_pupil = 1 - np.sum(resid**2, axis=1) / np.sum(V[:,itest]**2, axis=1)

In [ ]:
#### PREDICT USING BEHAVIOR PCs 
## regularized linear regression from behavior to neural PC's

# covariance of inputs
covM  = beh[:,itrain] @ beh[:,itrain].T
lam   = 1e6 # regularizer
covM  = covM + lam*np.eye(beh.shape[0]) # add the regularizer

# covariance between behavior and neural data
covMV = beh[:,itrain] @ V[:,itrain].T 

# we need to solve for x: covM * x = covMV
A     = np.linalg.solve(covM, covMV)

Vpred = A.T @ beh[:,itest]
resid = ? - ? # what are the unexplained residuals of the prediction? 

varexp = 1 - np.sum(resid**2, axis=1) / np.sum(V[:,itest]**2, axis=1)

In [ ]:
# plot of principal component activity and variance explained

fig=plt.figure(figsize=(12,3))

ipc = 0 ### which PC to plot

ax = fig.add_axes([0.05,.05,.75,.95])
ax.plot(V[ipc,itest])
ax.plot(Vpred[ipc])
ax.set_title('PC %d'%ipc)
ax.set_xlabel('time')
ax.set_ylabel('activity')

ax = fig.add_axes([0.9,.05, .2, .8])
ax.semilogx(np.arange(1,varexp.size+1), varexp, color='k')
ax.scatter(ipc+1, varexp[ipc],marker='x',color='r',s=200, lw=4, zorder=10)
ax.semilogx(np.arange(1,varexp.size+1), varexp_pupil, color=[0.,.5,0])
ax.text(1,0,'pupil',color=[0,.5,0])
ax.text(10,0.2,'motion SVD')
ax.set_xlabel('PC')
ax.set_ylabel('fraction variance explained')
ax.set_title('PC %d, varexp=%0.2f'%(ipc,varexp[ipc]))
plt.show()

# Sort neurons with rastermap

Let's see how well the behavioral prediction "looks". First let's sort the neural activity by rastermap, then use the sorting to sort the prediction of the neural activity.

In [ ]:
# make sorting of neurons with rastermap
from rastermap import Rastermap

# fit a rastermap model to S with one component and n_X=100, 
model = ??????

Now let's sort by rastermap and smooth over neurons

In [ ]:
# SMOOTHED NEURAL ACTIVITY
# this function performs a running average filter over the first dimension of X
def running_average(X, nbin = 100):
    Y = np.cumsum(X, axis=0)
    Y = Y[nbin:, :] - Y[:-nbin, :]
    return Y

# sort the neurons by the embedding dimension of the model
isort = ???embedding???

Sfilt = Sbin[isort,:] # resort Sbin
Sfilt = running_average(Sfilt) # smoothing
Sfilt = zscore(Sfilt, axis=1)

Let's sort the behavioral prediction by rastermap and smooth over neurons.

In [ ]:
# go from PC prediction to single neuron prediction
# U is the neurons by components PC matrix
# Vpred is our prediction of the components

Spred = ? @ ? # what is the prediction of the neurons? this matrix should number of neurons BY number of test points
Sfilt_pred = running_average(Spred[isort, :]) # re-sort the prediction according to the rastermap, and smooth
Sfilt_pred = zscore(Sfilt_pred, axis=1) 

In [ ]:
# make a rastermap of activity with prediction

fig = plt.figure(figsize=(16,12))
ax = fig.add_subplot(2,1,1)
ax.imshow(Sfilt[:, itest[:500]], vmin = -1, vmax=3, aspect='auto', cmap = 'gray_r')
ax.set_xlabel('time')
ax.set_ylabel('neurons')
ax.set_title('neural activity (test set)')

ax = fig.add_subplot(2,1,2)
ax.imshow(Sfilt_pred[:,:500], vmin = -1, vmax=3, aspect='auto', cmap = 'gray_r')
ax.set_xlabel('time')
ax.set_ylabel('neurons')
ax.set_title('behavior prediction (test set)')

plt.show()

In [ ]:
# how good is the prediction across the rastermap?

# Sfilt and Sfilt_pred contains neural activity/prediction smoothed across neurons.
# Pick every 50th row . It will contain the average of 100 neurons with similar activity. 
Sp      = ?????? # choose only testing timepoints to match the size of Sp_pred
Sp_pred = ?????? 

cc = np.zeros(nn)
for j in range(nn):
    C0 = np.corrcoef(Sp[j,:], Sp_pred[j,:])
    cc[j] = C0[0,1]

In [ ]:
# prediction quality across the rastermap

fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(1,1,1)
ax.plot(cc)
ax.set_ylim([0,1])
ax.set_ylabel('correlation btw/ pred and true')
ax.set_xlabel('depth on rastermap')
plt.show()